In [1]:
import nltk
import pymorphy2
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
import joblib
import random
import warnings

warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/tea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tea/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
morph = pymorphy2.MorphAnalyzer()


def lemmatize(text):
    words = text.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return lemmatized_words


def parseNews(news):
    regex = re.compile(r'[.|!|?|…]')
    lemmatized = []
    vocab = []
    stopWords = set(stopwords.words('russian'))

    for new in news:
        sentences = [t.strip() for t in regex.split(new)]
        lemSentences = []
        for s in sentences:
            words = [t for t in lemmatize(s) if t not in stopWords]
            lemSentences += words
            vocab += words
        lemmatized.append(lemSentences)

    popular = [k for k, v in sorted(dict(Counter(vocab)).items(), key=lambda item: item[1], reverse=True)]

    return lemmatized, popular[:512]


def getVectors(news, vocab):
    return [[1 if word in new else 0 for word in vocab] for new in news]

In [3]:
print('Reading news')
df = pd.read_csv('news.csv')
df

Reading news


,N,source,rubric,title,text
0,0,lenta.ru,Экономика,Синий богатырь,В 1930-е годы Советский Союз охватила лихорадк...
1,1,lenta.ru,Спорт,Загитова согласилась вести «Ледниковый период»,Олимпийская чемпионка по фигурному катанию Ал...
2,2,lenta.ru,Из жизни,Объяснена опасность однообразного питания,Российский врач-диетолог Римма Мойсенко объясн...
3,3,lenta.ru,Интернет и СМИ,«Предохраняться? А зачем?»,В 2019 году телеканал «Ю» запустил адаптацию з...
4,4,lenta.ru,Культура,Ефремов систематически употреблял наркотики,Актер Михаил Ефремов систематически употребл...
...,...,...,...,...,...
21668,21668,tjournal.ru,NaN,\n Россия прекратил...,\n\n \n \n \n \n \n...
21669,21669,tjournal.ru,NaN,\n Во Владивостоке ...,\n\n \n \n \n \n \n...
21670,21670,tjournal.ru,NaN,\n Дым от австралий...,\n\n \n \n \n \n \n...
21671,21671,tjournal.ru,NaN,\n Около 200 жителе...,\n\n \n \n \n \n \n...


In [4]:
print('Cleaning news')
df = df[df['rubric'].notnull()]
texts = df['text'].tolist()
labels = df['rubric'].tolist()

print('Lemmatizing')
lemmatized, lemmas = parseNews(texts)

joblib.dump(lemmas, 'vocab.joblib')

print('Calculating vectors')
vectors = getVectors(lemmatized, lemmas)

print(lemmatized[0][:5])

Cleaning news
Lemmatizing
Calculating vectors
['1930-й', 'год', 'советский', 'союз', 'охватить']


In [5]:
tags = list(set(labels))
print(tags)

print('Indexing labels')
y = np.array([tags.index(label) for label in labels])

joblib.dump(tags, 'tags.joblib')

['Из жизни', 'Наука и техника', 'Экономика', 'Ценности', 'Нацпроекты', 'Дом', 'Путешествия', '69-я параллель', 'Россия', 'Спорт', 'Интернет и СМИ', 'Культура', 'Силовые структуры', 'Бывший СССР', 'Мир']
Indexing labels


['tags.joblib']

In [6]:
xTrain, xTest, yTrain, yTest = train_test_split(vectors, y, test_size=0.2, random_state=42)

classifiers = [
    (GaussianNB(), 'bayes'),
    (DecisionTreeClassifier(), 'tree'),
    (LogisticRegression(), 'regression'),
    (SVC(), 'supportVectors')
]

print('training classifiers')

for classifier, name in classifiers:
    classifier.fit(xTrain, yTrain)

    predictions = classifier.predict(xTest)

    accuracy = accuracy_score(yTest, predictions)
    recall = recall_score(yTest, predictions, average='micro')

    print(f"Model: {name}")
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print()

    joblib.dump(classifier, f'models/{name.lower()}.joblib')

training classifiers
Model: bayes
Accuracy: 0.33962264150943394
Recall: 0.33962264150943394

Model: tree
Accuracy: 0.49500554938956715
Recall: 0.49500554938956715

Model: regression
Accuracy: 0.6725860155382908
Recall: 0.6725860155382908

Model: supportVectors
Accuracy: 0.6892341842397336
Recall: 0.6892341842397336



In [9]:
print('Getting sample file')
newData = df.iloc[random.sample(range(len(df)), 10)]
newData

Getting sample file


,N,source,rubric,title,text
2060,2060,lenta.ru,Россия,Поклонская впервые дала интервью украинскому ж...,"Депутат Госдумы , экс-прокурор Крыма Наталья..."
2824,2824,lenta.ru,Россия,В Подмосковье целая больница ушла на карантин ...,Губернатор Подмосковья Андрей Воробьев потре...
313,313,lenta.ru,Бывший СССР,Акция с огромным бело-красно-белым флагом в Гр...,В Гродно участники акции протеста растянули ог...
2827,2827,lenta.ru,Спорт,Возвращение российского футбола отложили,В Российском футбольном союзе (РФС) состояло...
4477,4477,lenta.ru,Силовые структуры,Россиянин убил коллегу на новогоднем корпорати...,В Пензе сотрудник центра занятости населения у...
1384,1384,lenta.ru,Из жизни,Пес знаменитой певицы спас ее новорожденного р...,Знаменитая ирландская певица Имельда Мэй расск...
797,797,lenta.ru,Бывший СССР,Силовики Украины задержали одного из лидеров ЛНР,Украинские правоохранители задержали одного из...
1753,1753,lenta.ru,Мир,Бунтующие из-за смерти чернокожего напали на ж...,Толпа людей напала на женщину в моторизованной...
415,415,lenta.ru,Бывший СССР,Советник Зеленского назвал необходимую для вос...,На восстановление Донбасса необходимо около 10...
2959,2959,lenta.ru,Россия,Собянин заявил о непройденном пике заболеваемо...,Пик заболеваемости коронавирусом еще не пройде...


In [10]:
realTags = list(newData['rubric'])
models = ['bayes', 'tree', 'regression', 'supportvectors']

vocab = joblib.load('vocab.joblib')
tags = joblib.load('tags.joblib')

lemmatized, _ = parseNews(newData['text'])
newVectors = getVectors(lemmatized, vocab)
trueValues = [tags.index(rub) for rub in realTags]

for model in models:
    alg = joblib.load('models/' + model + '.joblib')
    predictions = alg.predict(newVectors)
    print('model', model)
    print('Accuracy:', accuracy_score(trueValues, predictions))
    print('Recall:', recall_score(trueValues, predictions, average='micro'))
    if len(predictions) <= 10:
        for i, j in enumerate(predictions):
            print(f'{realTags[i]}: {tags[j]} {realTags[i] == tags[j]}')
    print()

model bayes
Accuracy: 0.5
Recall: 0.5
Россия: Силовые структуры False
Россия: Силовые структуры False
Бывший СССР: Бывший СССР True
Спорт: Спорт True
Силовые структуры: Силовые структуры True
Из жизни: Из жизни True
Бывший СССР: Силовые структуры False
Мир: Из жизни False
Бывший СССР: Бывший СССР True
Россия: Наука и техника False

model tree
Accuracy: 0.9
Recall: 0.9
Россия: Россия True
Россия: Россия True
Бывший СССР: Бывший СССР True
Спорт: Спорт True
Силовые структуры: Силовые структуры True
Из жизни: Из жизни True
Бывший СССР: Бывший СССР True
Мир: Мир True
Бывший СССР: Экономика False
Россия: Россия True

model regression
Accuracy: 1.0
Recall: 1.0
Россия: Россия True
Россия: Россия True
Бывший СССР: Бывший СССР True
Спорт: Спорт True
Силовые структуры: Силовые структуры True
Из жизни: Из жизни True
Бывший СССР: Бывший СССР True
Мир: Мир True
Бывший СССР: Бывший СССР True
Россия: Россия True

model supportvectors
Accuracy: 0.9
Recall: 0.9
Россия: Бывший СССР False
Россия: Россия T